# Trying out sympy module

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
import numpy as np
from Modules.sympy.classes import *
from Modules.sympy.utils import *
from sympy.physics.quantum import Dagger
from Modules.sympy.solver import *

In [3]:
Spin = RDBasis("\\sigma", 'spin', dim=2)
sx = Spin.project(sp.Matrix([[0, 1], [1, 0]]))
sz = Spin.project(sp.Matrix([[1, 0], [0, -1]]))

a = RDBoson("a", subspace ="b")
ad = RDBoson("{a^\\dagger}", subspace ="b", is_annihilation=False)

hbar = RDsymbol("hbar", order = 0)
omega = RDsymbol("omega", order = 0)
Omega_z = RDsymbol("\\Omega_{z}", order = 0)
g = RDsymbol("g", order = 1)

H = hbar * omega * ad * a + hbar * Omega_z * sp.Rational(1,2) * sz - hbar * g * (a + ad) * sx
H

\Omega_{z}*hbar*\sigma_{3}/2 - g*hbar*(a + {a^\dagger})*\sigma_{1} + hbar*omega*{a^\dagger}*a

In [4]:
from IPython.display import display, Math
def display_dict(d):
    for k, v in d.items():
        display(Math(f"{sp.latex(k)} = {sp.latex(v)}"))

In [5]:
S, H_diagonal, H_block = solver(H, Spin, order=2)


--------------------------------------------------
Order 1
Got ansatz
Solving
	Grouped by infinite operators
		Key: 1
		Got Solutions
		Key: a
		Got Solutions
		Key: {a^\dagger}
		Got Solutions
Got S_k_solved
Got tmp_H
Got H_final
Got Bk
--------------------------------------------------


--------------------------------------------------
Order 2
Got ansatz
Solving
	Grouped by infinite operators
		Key: 1
		Got Solutions
		Key: a**2
		Got Solutions
		Key: {a^\dagger}**2
		Got Solutions
Got S_k_solved
Got tmp_H
Got H_final
Got Bk
--------------------------------------------------



In [11]:
H_block

I*\Omega_{z}**3*g**3*hbar*a*\sigma_{2}/(\Omega_{z}**4*omega - 2*\Omega_{z}**2*omega**3 + omega**5) - I*\Omega_{z}**3*g**3*hbar*a**2*{a^\dagger}*\sigma_{2}/(\Omega_{z}**4*omega - 2*\Omega_{z}**2*omega**3 + omega**5) - I*\Omega_{z}**3*g**3*hbar*a**3*\sigma_{2}/(\Omega_{z}**4*omega - 2*\Omega_{z}**2*omega**3 + omega**5) + I*\Omega_{z}**3*g**3*hbar*{a^\dagger}*\sigma_{2}/(\Omega_{z}**4*omega - 2*\Omega_{z}**2*omega**3 + omega**5) + I*\Omega_{z}**3*g**3*hbar*{a^\dagger}**2*a*\sigma_{2}/(\Omega_{z}**4*omega - 2*\Omega_{z}**2*omega**3 + omega**5) + I*\Omega_{z}**3*g**3*hbar*{a^\dagger}**3*\sigma_{2}/(\Omega_{z}**4*omega - 2*\Omega_{z}**2*omega**3 + omega**5) - 2*\Omega_{z}**3*g**2*hbar*a*{a^\dagger}*\sigma_{3}/(\Omega_{z}**4 - 2*\Omega_{z}**2*omega**2 + omega**4) + \Omega_{z}**3*g**2*hbar*\sigma_{3}/(\Omega_{z}**4 - 2*\Omega_{z}**2*omega**2 + omega**4) - 2*\Omega_{z}**2*g**3*hbar*a*{a^\dagger}**2*\sigma_{1}/(\Omega_{z}**4 - 2*\Omega_{z}**2*omega**2 + omega**4) - 8*\Omega_{z}**2*g**3*hbar*a*\s

# Extra

In [6]:
def create_equations_by_order(H, order=2):
    max_order = max(list(group_by_order(H).keys()))

    terms = [RDsymbol(f'\\varepsilon^{{({k})}}', order=k)*RDOperator(f'H_{{{k}}}', dim=2, subspace='RD', matrix=sp.eye(2)) for k in range(1, max_order+1)]
    H_aux = RDOperator(f'H_{{{0}}}', dim=2, subspace='RD', matrix=sp.eye(2))  + sum(terms)
    

    s_symbols = [RDsymbol(f'\\varepsilon^{{({k})}}', order=k) for k in range(1, order+1)]
    s_terms = [s_symbols[k-1]*RDOperator(f'S_{{{k}}}', dim=2, subspace='RD', matrix=sp.eye(2)) for k in range(1, order+1)]
    S = sum(s_terms)

    expanded_equations = 0
    for k in range(order+1):
        expanded_equations += (sp.Rational(1, sp.factorial(k)) * expand_commutator(nested_commutator(H_aux, S, k))).expand()
    
    terms = expanded_equations.as_ordered_terms()
    factors = [sp.Mul(*term.as_ordered_factors()[:-1]) for term in terms]
    
    subs_epsilons = {s_k: 1 for s_k in s_symbols}

    factors_order = {}
    for i, f in enumerate(factors):
        order_dict = group_by_order(f)
        if not isinstance(order_dict, dict):
            order_index = order_dict
            if order_index > order:
                continue
            factors_order[order_index] = factors_order.get(order_index, 0) + terms[i].subs(subs_epsilons)
            continue
        order_index = list(order_dict.keys())[0]
        if order_index > order:
            continue
        factors_order[order_index] = factors_order.get(order_index, 0) + terms[i].subs(subs_epsilons)
        
    return factors_order